In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
!pip install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.8 MB/s eta 0:00:00
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.9/366.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 18.9 MB/s eta 0:00:00
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.p

In [3]:
!rm -rf mmsegmentation
!git clone -b main https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 14740, done.
remote: Counting objects: 100% (327/327), done.
remote: Compressing objects: 100% (244/244), done.
remote: Total 14740 (delta 105), reused 191 (delta 76), pack-reused 14413
Receiving objects: 100% (14740/14740), 20.09 MiB | 28.06 MiB/s, done.
Resolving deltas: 100% (10408/10408), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation


In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

2.0.0+cu118 True
1.0.0


In [5]:
!mim download mmsegmentation --config pidnet-s_2xb6-120k_1024x1024-cityscapes --dest .

processing pidnet-s_2xb6-120k_1024x1024-cityscapes...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MiB 14.4 MB/s eta 0:00:00
Successfully downloaded pidnet-s_2xb6-120k_1024x1024-cityscapes_20230302_191700-bb8e3bcc.pth to /content/mmsegmentation
Successfully dumped pidnet-s_2xb6-120k_1024x1024-cityscapes.py to /content/mmsegmentation


In [6]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
config_file = '/content/mmsegmentation/pidnet-s_2xb6-120k_1024x1024-cityscapes.py'
checkpoint_file = '/content/mmsegmentation/pidnet-s_2xb6-120k_1024x1024-cityscapes_20230302_191700-bb8e3bcc.pth'

In [7]:
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

/content/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: /content/mmsegmentation/pidnet-s_2xb6-120k_1024x1024-cityscapes_20230302_191700-bb8e3bcc.pth


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p "/content/drive/MyDrive/Autonomous Driving/front_classifier/data/k360_pidnet/val/straight"
!mkdir -p "/content/drive/MyDrive/Autonomous Driving/front_classifier/data/k360_pidnet/val/threeway_left"
!mkdir -p "/content/drive/MyDrive/Autonomous Driving/front_classifier/data/k360_pidnet/val/threeway_right"
!mkdir -p "/content/drive/MyDrive/Autonomous Driving/front_classifier/data/k360_pidnet/val/threeway_t"
!mkdir -p "/content/drive/MyDrive/Autonomous Driving/front_classifier/data/k360_pidnet/val/fourway"

In [10]:
import glob
import os
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

# 0=straight, 1=left T, 2=right T, 3=left/right T, 4=cross
for filename in glob.glob("/content/drive/MyDrive/mobilenet/k360/validation/*/*.png"):
  im = Image.open(filename).resize((1024, 1024))
  im = np.array(im)
  result = inference_model(model, im)
  img = show_result_pyplot(model, im, result, show=False, opacity=1.0)
  red, green, blue = img.T # Temporarily unpack the bands for readability

  car_area = (red == 0) & (green == 0) & (blue == 142)
  truck_area = (red == 0) & (green == 0) & (blue == 70)
  bus_area = (red == 0) & (green == 60) & (blue == 100)
  caravan_area = (red == 0) & (green == 0) & (blue == 90)
  trailer_area = (red == 0) & (green == 0) & (blue == 110)
  train_area = (red == 0) & (green == 80) & (blue == 100)
  motorcycle_area = (red == 0) & (green == 0) & (blue == 230)
  bicycle_area = (red == 119) & (green == 11) & (blue == 32)
  road_area = (red == 128) & (green == 64) & (blue == 128)
  pedestrian_area = (red == 220) & (green == 20) & (blue == 60)
  img[:,:,:3] = (0, 255, 0)
  img[:,:,:3][car_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][truck_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][bus_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][caravan_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][trailer_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][train_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][motorcycle_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][bicycle_area.T] = (0, 0, 255) # Transpose back needed
  img[:,:,:3][pedestrian_area.T] = (0, 0, 255)
  img[:,:,:3][road_area.T] = (255, 0, 0) # Transpose back needed

  im = Image.fromarray(img)
  path = filename.replace("mobilenet/k360/", "Autonomous Driving/front_classifier/data/k360_pidnet/")
  path = path.replace("/validation/", "/val/")
  path = path.replace("/0/", "/straight/")
  path = path.replace("/1/", "/threeway_left/")
  path = path.replace("/2/", "/threeway_right/")
  path = path.replace("/3/", "/threeway_t/")
  path = path.replace("/4/", "/fourway/")
  im.save(path)
